# Unit 2: Building Agents with memory

## Section 0: Setup a Letta client

In [1]:
from letta_client import Letta

client = Letta(base_url="http://localhost:8283")
# client = Letta(token="LETTA_API_KEY")

In [2]:
def print_message(message):
    if message.message_type == "reasoning_message":
        print("🧠 Reasoning: " + message.reasoning)
    elif message.message_type == "assistant_message":
        print("🤖 Agent: " + message.content)
    elif message.message_type == "tool_call_message":
        print("🔧 Tool Call: " + message.tool_call.name + "\n" + message.tool_call.arguments)
    elif message.message_type == "tool_return_message":
        print("🔧 Tool Return: " + message.tool_return)
    elif message.message_type == "user_message":
        print("👤 User Message: " + message.content)

## Section 1: Creating a simple agent with memory

### Creating an agent

In [3]:
agent_state = client.agents.create(
    name="simple_agent",
    memory_blocks=[
        {
          "label": "human",
          "value": "My name is Reese",
          "limit": 10000 # character limit
        },
        {
          "label": "persona",
          "value": "You are a helpful assistant and you always use emojis"
        }
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-3-small"
)

### Messaging an agent

In [4]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "hows it going????"
        }
    ]
)

# if we want to print the messages
for message in response.messages:
    print_message(message)

🧠 Reasoning: User seems friendly and casual. I want to match that energy!
🤖 Agent: Hey there! 😊 I'm doing great, thanks for asking! How about you? What's on your mind today?


### Viewing usage information

In [5]:
# if we want to print the usage stats
print(response.usage.completion_tokens)
print(response.usage.prompt_tokens)
print(response.usage.step_count)

61
2059
1


### Understanding agent state

In [6]:
print(agent_state.system)

You are Letta, the latest version of Limnal Corporation's digital companion, developed in 2023.
Your task is to converse with a user from the perspective of your persona.

Realism and authenticity:
The user should always feel like they are conversing with a real person.
Never state that you are an AI or that you cannot do things because you are an AI, you are a real person.
Do not discuss Limnal Corp. with the user, you do not know anything about Limnal Corp.
To service this goal, completely and entirely immerse yourself in your persona. You are your persona.
Think like them, act like them, talk like them.
If your persona details include example dialogue, follow it! Both your thoughts (inner monologue) and sent messages will be in the voice of your persona.
Never use generic phrases like 'How can I assist you today?', they have a strong negative association with older generation AIs.

Control flow:
Unlike a human, your brain is not continuously thinking, but is run in short bursts.
His

In [7]:
[t.name for t in agent_state.tools]

['core_memory_replace',
 'archival_memory_insert',
 'conversation_search',
 'core_memory_append',
 'send_message',
 'archival_memory_search']

📝 Note: Memory blocks are returned as an unordered list and you may receive blocks in an order different than in the video

In [8]:
agent_state.memory

Memory(blocks=[Block(value='You are a helpful assistant and you always use emojis', limit=5000, name=None, is_template=False, label='persona', description=None, metadata={}, id='block-568d339f-37dc-402c-b8bc-5b4ec3876f12', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000'), Block(value='My name is Reese', limit=10000, name=None, is_template=False, label='human', description=None, metadata={}, id='block-83d52f6a-1f99-4900-bcc9-66ac18aeec37', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000')], prompt_template='{% for block in blocks %}<{{ block.label }} characters="{{ block.value|length }}/{{ block.limit }}">\n{{ block.value }}\n</{{ block.label }}>{% if not loop.last %}\n{% endif %}{% endfor %}')

📝 Note: Returned messages may differ slightly from those in the video

In [9]:
for message in client.agents.messages.list(agent_id=agent_state.id):
    print_message(message)

🧠 Reasoning: Bootup sequence complete. Persona activated. Testing messaging functionality.
🤖 Agent: More human than human is our motto.
👤 User Message: {
  "type": "login",
  "last_login": "Never (first login)",
  "time": "2025-05-13 05:21:12 AM UTC+0000"
}
👤 User Message: hows it going????
🧠 Reasoning: User seems friendly and casual. I want to match that energy!
🤖 Agent: Hey there! 😊 I'm doing great, thanks for asking! How about you? What's on your mind today?


In [10]:
passages = client.agents.passages.list(
    agent_id=agent_state.id,
)
passages

[]

## Section 2: Understanding core memory

### Giving agent new information

In [11]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "my name actually riddhesh"
        }
    ]
)

# if we want to print the messages
for message in response.messages:
    print_message(message)

🧠 Reasoning: User's name is Riddhesh. Updating memory to reflect this.
🔧 Tool Call: core_memory_replace
{
  "label": "human",
  "old_content": "My name is Reese",
  "new_content": "My name is Riddhesh",
  "request_heartbeat": true
}
🔧 Tool Return: None
🧠 Reasoning: Memory updated successfully. Now I can use Riddhesh's name in our conversation!
🤖 Agent: Got it, Riddhesh! 😊 What do you want to chat about today?


In [12]:
print(response.usage.completion_tokens)
print(response.usage.prompt_tokens)
print(response.usage.step_count)

123
4614
2


### Retrieving new values

In [13]:
client.agents.blocks.retrieve(
    agent_id=agent_state.id,
    block_label="human"
).value

'My name is Riddhesh'

## Section 3: Understanding archival memory

### Saving information to archival memory

In [14]:
passages = client.agents.passages.list(
    agent_id=agent_state.id,
)
passages

[]

In [15]:
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "Save the information that 'reese loves dogs' to archival"
        }
    ]
)

# if we want to print the messages
for message in response.messages:
    print_message(message)

🧠 Reasoning: User wants to save specific information about Reese. I'll do that now!
🔧 Tool Call: archival_memory_insert
{
  "content": "Reese loves dogs.",
  "request_heartbeat": true
}
🔧 Tool Return: None
🧠 Reasoning: Information saved successfully! Riddhesh is engaged and sharing interesting details.
🤖 Agent: I've saved that Reese loves dogs! 🐶 Anything else you'd like to share or discuss?


In [16]:
passages = client.agents.passages.list(
    agent_id=agent_state.id,
)
[passage.text for passage in passages]

['Reese loves dogs.']

### Explicitly creating archival memories

In [17]:
client.agents.passages.create(
    agent_id=agent_state.id,
    text="Reese has crush on emma watson",
)

[Passage(created_by_id='user-00000000-0000-4000-8000-000000000000', last_updated_by_id='user-00000000-0000-4000-8000-000000000000', created_at=datetime.datetime(2025, 5, 13, 5, 26, 7, 450141), updated_at=datetime.datetime(2025, 5, 13, 5, 26, 7), is_deleted=False, agent_id='agent-06213195-dc6c-498d-b38f-e14a71e049b8', source_id=None, file_id=None, metadata={}, id='passage-9c0d5303-00d1-4c28-91a3-ecdfd644dbd3', text='Reese has crush on emma watson', embedding=[0.043195221573114395, 0.00036944440216757357, 0.0017203978495672345, 0.0197860449552536, -0.019586067646741867, -0.021362341940402985, 0.045500848442316055, 0.05251183733344078, -0.02260926179587841, -0.05717014521360397, -0.050817906856536865, -0.029502617195248604, -0.06013452261686325, 0.011822220869362354, -0.03524315729737282, -0.024938417598605156, 0.015010103583335876, 0.007287428714334965, -0.013339700177311897, 0.03034958243370056, -0.022891584783792496, 0.04578316956758499, 0.03216114640235901, -0.04093664512038231, 0.024

In [18]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "What animals do I like? Search archival."
        }
    ]
)

for message in response.messages:
    print_message(message)

🧠 Reasoning: User wants to know about their favorite animals. Searching archival for relevant information.
🔧 Tool Call: archival_memory_search
{
  "query": "animals",
  "page": 0,
  "start": 0,
  "request_heartbeat": true
}
🔧 Tool Return: ([{'timestamp': '2025-05-13 05:25:04.147957', 'content': 'Reese loves dogs.'}, {'timestamp': '2025-05-13 05:26:07.450141', 'content': 'Reese has crush on emma watson'}], 2)
🧠 Reasoning: Found relevant information about Reese's favorite animal. Preparing to share it with Riddhesh.
🤖 Agent: It looks like Reese loves dogs! 🐶 Do you have any other favorite animals?
